In [1]:
from nltk.corpus import stopwords, wordnet
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string as s
import numpy as np
import pandas as pd

In [2]:
# loading training and testing data
train = pd.read_csv("twitter_x_y_train.csv")
test = pd.read_csv("twitter_x_test.csv")

In [3]:
# printing first five training data
train.head()

tweet_id airline_sentiment    airline airline_sentiment_gold  \
0  567900433542488064          negative  Southwest                    NaN   
1  569989168903819264          positive  Southwest                    NaN   
2  568089179520954368          positive     United                    NaN   
3  568928195581513728          negative  Southwest                    NaN   
4  568594180014014464          negative     United                    NaN   

            name negativereason_gold  retweet_count  \
0  ColeyGirouard                 NaN              0   
1  WalterFaddoul                 NaN              0   
2      LocalKyle                 NaN              0   
3    amccarthy19                 NaN              0   
4        J_Okayy                 NaN              0   

                                                text tweet_coord  \
0  @SouthwestAir I am scheduled for the morning, ...         NaN   
1  @SouthwestAir seeing your workers time in and ...         NaN   
2  @united Flew ORD to Miami and back and  had gr...         NaN   
3     @SouthwestAir @dultch97 that's horse radish 😤🐴         NaN   
4  @united so our flight into ORD was delayed bec...         NaN   

               tweet_created              tweet_location  \
0  2015-02-17 20:16:29 -0800             Washington D.C.   
1  2015-02-23 14:36:22 -0800  Indianapolis, Indiana; USA   
2  2015-02-18 08:46:29 -0800                    Illinois   
3  2015-02-20 16:20:26 -0800                         NaN   
4  2015-02-19 18:13:11 -0800                         NaN   

                user_timezone  
0      Atlantic Time (Canada)  
1  Central Time (US & Canada)  
2  Central Time (US & Canada)  
3      Atlantic Time (Canada)  
4  Eastern Time (US & Canada)

In [4]:
train.describe(include='all')

tweet_id airline_sentiment airline airline_sentiment_gold  \
count   1.098000e+04             10980   10980                     31   
unique           NaN                 3       6                      3   
top              NaN          negative  United               negative   
freq             NaN              6851    2928                     24   
mean    5.692169e+17               NaN     NaN                    NaN   
std     7.795438e+14               NaN     NaN                    NaN   
min     5.675883e+17               NaN     NaN                    NaN   
25%     5.685584e+17               NaN     NaN                    NaN   
50%     5.694753e+17               NaN     NaN                    NaN   
75%     5.698902e+17               NaN     NaN                    NaN   
max     5.703106e+17               NaN     NaN                    NaN   

               name     negativereason_gold  retweet_count            text  \
count         10980                      24   10980.000000           10980   
unique         6438                      11            NaN           10851   
top     JetBlueNews  Customer Service Issue            NaN  @united thanks   
freq             43                       9            NaN               6   
mean            NaN                     NaN       0.080965             NaN   
std             NaN                     NaN       0.740303             NaN   
min             NaN                     NaN       0.000000             NaN   
25%             NaN                     NaN       0.000000             NaN   
50%             NaN                     NaN       0.000000             NaN   
75%             NaN                     NaN       0.000000             NaN   
max             NaN                     NaN      44.000000             NaN   

       tweet_coord              tweet_created tweet_location  \
count          776                      10980           7430   
unique         632                      10758           2658   
top     [0.0, 0.0]  2015-02-24 11:43:05 -0800   New York, NY   
freq           131                          3            125   
mean           NaN                        NaN            NaN   
std            NaN                        NaN            NaN   
min            NaN                        NaN            NaN   
25%            NaN                        NaN            NaN   
50%            NaN                        NaN            NaN   
75%            NaN                        NaN            NaN   
max            NaN                        NaN            NaN   

                     user_timezone  
count                         7403  
unique                          78  
top     Eastern Time (US & Canada)  
freq                          2819  
mean                           NaN  
std                            NaN  
min                            NaN  
25%                            NaN  
50%                            NaN  
75%                            NaN  
max                            NaN

# For applying NLTK, we need only 'text' attibute as x-train dataset for training with 'airline_sentiment' attribute as y-train. Also 'text' attribute as x-test

In [5]:
x_train = train.pop("text")
y_train = train.pop("airline_sentiment")
x_test = test.pop("text")

In [6]:
# stopwords
stop = stopwords.words('english') + list(s.punctuation)

In [7]:
# lemmatization
lemmatizer = WordNetLemmatizer()

In [8]:
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('R'):
        return wordnet.ADV
    elif tag.startswith('V'):
        return wordnet.VERB
    else:
        return wordnet.NOUN

In [9]:
def clean_words(sentence):
    output = []
    for w in word_tokenize(sentence):
        if w.lower() not in stop:
            pos_t = pos_tag(w)[0][1]
            clean_w = lemmatizer.lemmatize(w, pos=get_simple_pos(pos_t))
            output.append(clean_w.lower())
    return output

In [10]:
x_train = [clean_words(sent) for sent in x_train]
x_test = [clean_words(sent) for sent in x_test] 

In [11]:
x_train = [" ".join(w) for w in x_train]
x_test = [" ".join(w) for w in x_test]

In [12]:
# using Tfidf Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features = 2000)
x_train_vec = vectorizer.fit_transform(x_train)
x_test_vec = vectorizer.transform(x_test)

In [17]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_train_vec, y_train)
y_pred = clf.predict(x_test_vec)
print(clf.score(x_train_vec, y_train))

0.7819672131147541


In [18]:
np.savetxt("Predictions_twitter_sentiments.csv", y_pred, fmt="%s")